In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path1 = '/content/drive/MyDrive/AIO/transferlearning/data50/train'
file_path2 = '/content/drive/MyDrive/AIO/transferlearning/data50/validation'

In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision
import torchvision.models as models

torch.manual_seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),   # Resize images to a fixed size
        transforms.ToTensor(),           # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.75,
                                 scale=(0.01, 0.3),
                                 ratio=(1.0, 1.0),
                                 value=0,
                                 inplace =True)
    ])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize images to a fixed size
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) # Normalize
])

# Load datasets
train_dataset = datasets.ImageFolder(file_path1, transform=train_transform)
test_dataset = datasets.ImageFolder(file_path2, transform=test_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
from torchsummary import summary

vgg16 = models.vgg16()
f_extractor = vgg16.features

model = nn.Sequential(
    f_extractor,
    nn.Flatten(),
    nn.Dropout(0.3),
    nn.Linear(512* 7*7, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 2)
)

model.to(device)
summary(model,(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

In [ ]:
# Function to compute loss and accuracy for test set
def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    running_correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            # Move inputs and labels to the device
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            running_correct += (predicted == labels).sum().item()

    accuracy = 100 * running_correct / total
    test_loss = test_loss / len(test_loader)
    return test_loss, accuracy

In [ ]:
# some parameter
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
max_epoch = 50

In [ ]:
# train
for epoch in range(max_epoch):
    model.train()
    running_loss = 0.0
    running_correct = 0   # to track number of correct predictions
    total = 0             # to track total number of samples

    for i, (inputs, labels) in enumerate(train_loader, 0):
        # Move inputs and labels to the device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Determine class predictions and track accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        running_correct += (predicted == labels).sum().item()


    epoch_accuracy = 100 * running_correct / total
    epoch_loss = running_loss / (i + 1)

    test_loss, test_accuracy = evaluate(model, test_loader, criterion)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

Epoch [1/50], Loss: 0.6919, Accuracy: 50.00%, Test Loss: 0.6973, Test Accuracy: 50.00%
Epoch [2/50], Loss: 0.6859, Accuracy: 58.00%, Test Loss: 0.6861, Test Accuracy: 62.00%
Epoch [3/50], Loss: 0.6770, Accuracy: 69.00%, Test Loss: 0.6832, Test Accuracy: 59.00%
Epoch [4/50], Loss: 0.6392, Accuracy: 61.00%, Test Loss: 0.7342, Test Accuracy: 55.00%
Epoch [5/50], Loss: 0.6048, Accuracy: 70.00%, Test Loss: 0.6270, Test Accuracy: 57.00%
Epoch [6/50], Loss: 0.7600, Accuracy: 67.00%, Test Loss: 0.7782, Test Accuracy: 57.00%
Epoch [7/50], Loss: 0.5810, Accuracy: 70.00%, Test Loss: 0.7676, Test Accuracy: 48.00%
Epoch [8/50], Loss: 0.6200, Accuracy: 75.00%, Test Loss: 0.8138, Test Accuracy: 56.00%
Epoch [9/50], Loss: 0.5205, Accuracy: 75.00%, Test Loss: 0.7905, Test Accuracy: 55.00%
Epoch [10/50], Loss: 0.6146, Accuracy: 69.00%, Test Loss: 0.7275, Test Accuracy: 56.00%
Epoch [11/50], Loss: 0.8175, Accuracy: 75.00%, Test Loss: 0.7100, Test Accuracy: 57.00%
Epoch [12/50], Loss: 0.5816, Accuracy: 68